# Suspicious PsExec execution
detects execution of psexec or paexec with renamed service name, this rule helps to filter out the noise if psexec is used for legit purposes or if attacker uses a different psexec client other than sysinternal one

## Rule Content
```
- title: Suspicious PsExec execution
  id: c462f537-a1e3-41a6-b5fc-b2c2cef9bf82
  description: detects execution of psexec or paexec with renamed service name, this
    rule helps to filter out the noise if psexec is used for legit purposes or if
    attacker uses a different psexec client other than sysinternal one
  author: Samir Bousseaden
  references:
  - https://blog.menasec.net/2019/02/threat-hunting-3-detecting-psexec.html
  tags:
  - attack.lateral_movement
  - attack.t1077
  logsource:
    product: windows
    service: security
    description: The advanced audit policy setting "Object Access > Audit Detailed
      File Share" must be configured for Success/Failure
    category: null
  detection:
    selection1:
      EventID: 5145
      ShareName: \\*\IPC$
      RelativeTargetName:
      - '*-stdin'
      - '*-stdout'
      - '*-stderr'
    selection2:
      EventID: 5145
      ShareName: \\*\IPC$
      RelativeTargetName: PSEXESVC*
    condition: selection1 and not selection2
  falsepositives:
  - nothing observed so far
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((event_id:"5145" AND share_name.keyword:\\*\\IPC$ AND share_relative_target_name.keyword:(*\-stdin OR *\-stdout OR *\-stderr)) AND (NOT (event_id:"5145" AND share_name.keyword:\\*\\IPC$ AND share_relative_target_name.keyword:PSEXESVC*)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()